In [ ]:
#setup
!pip install spacy
!pip install transformers

To make this notebook work you need conda and do this in your environment: conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

In [ ]:
#Datamodell for Spacy
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
from transformers import pipeline

In [ ]:
def computeSentimentOfEntities(sentence):
    sentiment_analyzer = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(sentence)
    entity_sentiments = {}

    # Analyze sentiment of each entity and its context
    for ent in doc.ents:
        if ent.label_ in {"PERSON", "ORG"}:
            # Extract surrounding context for the entity
            entity_context = sentence[ent.start_char: ent.end_char + 20]
            # Run the sentiment analysis on entity context
            sentiment = sentiment_analyzer(entity_context)[0]
            entity_sentiments[ent.text] = sentiment

    return entity_sentiments

In [ ]:
label_mapping = {
    "LABEL_0": "negative",
    "LABEL_1": "neutral",
    "LABEL_2": "positive"
}

In [ ]:
def labelToReadableLabelConverter(entitySentiment):
    # Define a mapping for labels to sentiment
    interpreted_results = {}
    for entity, result in entitySentiment.items():
        label = result["label"]
        interpreted_label = label_mapping.get(label, "unknown")  # map label to sentiment
        interpreted_results[entity] = {
            "sentiment": interpreted_label,
            "score": result["score"]
        }
    return interpreted_results


    

In [ ]:
#tests
tests = list()
tests.append("Harris is pretty good")
tests.append("Harris is the worst")
tests.append("Kamala Harris sucks")
tests.append("Donald Trump is a dump")
tests.append("Trump is the stupidest man i ever encountered, he is just a stupid dick with shit comments and bullshits all over the place")
tests.append("I love Trump and i want him to be president forever")
tests.append("Kamala and Trump are both horrible")
tests.append("I think Kamala and Trump are both good")
tests.append(" fuck this system. Vote for Trump")
for test in tests:
    analysis = computeSentimentOfEntities(test)
    print(labelToReadableLabelConverter(analysis))
    